In [ ]:
# read mails from files and store in array mails

import csv
import os
import re
import sys
import xml.etree.ElementTree as ET


INFILENAMES = [ "../usb/OVK/data/eriktks/AS/text/AS-mails.csv","../usb/OVK/data/eriktks/ES/text/ES-mails.csv" ]
CLIENT = "CLIENT"
COUNSELOR = "counselor"
ID = "client-id"
NBROFCHARS = "nbrOfCharsInWords"
NBROFSENTS = "nbrOfSents"
NBROFWORDS = "nbrOfWords"
SENDER = "sender"
SEPARATOR = ","
TEXT = "text"

mails = []
senders = []
ids = []
nbrOfChars = []
counselors = []
for inFileName in INFILENAMES:
    try: inFile = open(inFileName,"r")
    except: sys.exit("cannot read file "+INFILENAME)
    csvReader = csv.DictReader(inFile,delimiter=SEPARATOR)
    for row in csvReader:
        try:
            if True:
                mails.append(row[TEXT])
                senders.append(row[SENDER])
                nbrOfChars.append(int(row[NBROFCHARS]))
                counselors.append(row[COUNSELOR])
                ids.append(row[ID]+","+row[COUNSELOR]+","+row[NBROFWORDS]+","+row[NBROFCHARS]+","+row[NBROFSENTS]+","+row[SENDER])
        except: sys.exit("unexpected row in file "+INFILENAME+": "+str(row))
    inFile.close()

In [ ]:
# create tfidf vectors for each mail text

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8,max_features=200000, \
                                   min_df=0.2,use_idf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(mails)

In [ ]:
# convert tfidf vectors to principal components (n=2)

from sklearn.decomposition import PCA

pca = PCA(n_components=4)
pca.fit(tfidf_matrix.toarray())
newSpace = pca.transform(tfidf_matrix.toarray())

In [ ]:
# draw graph of pca data: red: from client; blue: from counselor

%matplotlib notebook
import matplotlib.pyplot as plt
import random
import re

EXPERIMENT = "Emails"
THRESHOLD = 5
RANDOMFACTOR=0.00
xSmall = []
ySmall = []
idsSmall = []
xLarge = []
yLarge = []
idsLarge = []
random.seed()
for i in range(0,len(newSpace)):
    if senders[i] == CLIENT:
        xSmall.append(newSpace[i][0]+random.random()*RANDOMFACTOR)
        ySmall.append(newSpace[i][1]+random.random()*RANDOMFACTOR)
        idsSmall.append(ids[i])
    else:
        xLarge.append(newSpace[i][0]+random.random()*RANDOMFACTOR)
        yLarge.append(newSpace[i][1]+random.random()*RANDOMFACTOR)
        idsLarge.append(ids[i])
fig = plt.figure(figsize=(9,5))
plt.gca().set_title(EXPERIMENT+": Click in the graph to see the relevant document ids")
largeDots = plt.scatter(xLarge,yLarge,s=1,color="blue",picker=2,label="from counselor".format(THRESHOLD))
smallDots = plt.scatter(xSmall,ySmall,s=1,color="red",picker=2,label="from client".format(THRESHOLD))
unknownDots = -1
plt.legend(fontsize=8)

def pickScatter(event):
    dataIds = []
    for i in range(0,len(event.ind)): 
        if event.artist == smallDots: dataIds.append(idsSmall[event.ind[i]])
        else: dataIds.append(idsLarge[event.ind[i]])
    plt.gca().set_title(EXPERIMENT+str(dataIds),fontsize=12)
plt.gcf().canvas.mpl_connect("pick_event",pickScatter)